In [ ]:
!pip install langchain_community
!pip install langchain
!pip install langchain-mistralai
!pip install python-dotenv
!pip install requests

In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import requests
import json
load_dotenv()

In [ ]:
## HELPERS

def get_prompt_txt(prompt_path) -> str:
    prompt_txt = ""
    with open(
        prompt_path, "r"
    ) as f:
        prompt_txt = f.read()  # has

    
    return prompt_txt



In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed
import ast
import json

model = ChatMistralAI(model="open-mixtral-8x7b", temperature=0.7)
output_parser = StrOutputParser()


# QUERY Optimization
query_optimization_prompt = get_prompt_txt("../prompts/query_optimization.txt")

optimization_prompt_template =  ChatPromptTemplate.from_messages(
            [("human", query_optimization_prompt)],
        )
optimization_chain = optimization_prompt_template | model | output_parser


# Summarization

res_summary_prompt = get_prompt_txt("../prompts/result_summarization.txt")

res_summary_prompt_template =  ChatPromptTemplate.from_messages(
            [("human", res_summary_prompt)],
        )
summarization_chain = res_summary_prompt_template | model | output_parser




In [ ]:
#1. optimize
@retry(stop=stop_after_attempt(10), wait=wait_fixed(1))
def optimize_query(query: str) -> dict:
    raw_asnwer = optimization_chain.invoke({"q": query})
    
    llm_answer = ast.literal_eval(raw_asnwer)
    dict_ = json.loads(llm_answer) if isinstance(llm_answer, str) else llm_answer
    return dict_


#2. Call MOSAIC with optimizes query
optimized_query = "What is the largest country in Europe?"
url = f"https://qnode.eu/ows/mosaic/service/search?q={optimized_query}"

try:
    response = requests.get(url)
    response.raise_for_status()
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
except requests.exceptions.HTTPError as http_err: 
    print(f"HTTP error occurred: {http_err}") 
except Exception as err: 
    print(f"Other error occurred: {err}") 


#3. get MOSAIC REsults and feed it to summarization_chain




In [ ]:


 
 #1. optimize
query_optimized = optimize_query("climate change")
query_optimized

results = {}
results["clarified_query"] = []# search_mosaic(query=query_optimized["clarified_query"])
for i, suggested_queries in enumerate(query_optimized['subqueries']):
    results[f"subquery_{i}"] = []# search_mosaic(query=suggested_queries)


#2. call mosai with res


#3. get MOSAIC REsults and feed it to summarization_chain



SyntaxError: invalid syntax (2334311790.py, line 1)